In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
breed_df = pd.read_excel('dog_breed_list.xlsx', names=['Breed'])
breed_df.head()

,Breed
0,affenpinscher
1,american_staffordshire_terrier
2,australian_terrier
3,basset
4,bernese_mountain_dog


In [3]:
breed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Breed   36 non-null     object
dtypes: object(1)
memory usage: 416.0+ bytes


In [4]:
breed_list = list(breed_df['Breed'])
breed_list = [breed.replace('_', ' ').title().replace(' ', '_') for breed in breed_list]
breed_df['Name_in_Wiki'] = breed_list

In [5]:
breed_df['Name_in_Wiki'][3] = 'Basset_Hound'
breed_df['Name_in_Wiki'][8] = 'Boxer_(dog)'
breed_df['Name_in_Wiki'][10] = 'Brittany_(dog)'
breed_df['Name_in_Wiki'][11] = 'Bullmastiff'
breed_df['Name_in_Wiki'][17] = 'German_Shorthaired_Pointer'
breed_df['Name_in_Wiki'][34] = 'English_Toy_Terrier_(Black_%26_Tan)'

breed_df

,Breed,Name_in_Wiki
0,affenpinscher,Affenpinscher
1,american_staffordshire_terrier,American_Staffordshire_Terrier
2,australian_terrier,Australian_Terrier
3,basset,Basset_Hound
4,bernese_mountain_dog,Bernese_Mountain_Dog
5,bloodhound,Bloodhound
6,border_collie,Border_Collie
7,border_terrier,Border_Terrier
8,boxer,Boxer_(dog)
9,briard,Briard


In [6]:
wiki_url = 'https://en.wikipedia.org/wiki/'

In [7]:
scraping_data = []

for breed in breed_df['Name_in_Wiki']:
    url = wiki_url + breed
    page = requests.get(url)

    dict = {'Breed': breed}
 
    soup = BeautifulSoup(page.text, 'lxml')

    infobox = soup.find('table', {'class': 'infobox'})

    try:
        origin = infobox.find('th', text='Origin').findNextSibling('td').text

        if origin is not None:
            dict['Origin'] = origin
        else:
            print(f"There are NO origin data for {breed}")

    except AttributeError as s:
        print("AttributeError occured: ", s)



    try:
        table = soup.find('table', {'class': 'infobox-subbox'})

        if table is not None:
            header = []
            for i in table.find_all('th')[1:]:
                title = i.text.replace("\n", "").replace("\xa0", "")
                header.append(title)

            data = []

            for j in table.find_all('tr')[1:]:
                rows = j.find_all('td')
                row_data = [row.text.replace("\n", "").replace("\xa0", "") for row in rows]
                data.append(row_data)


            if len(header) == len(data):
                for i in range(len(header)):
                    dict[header[i]] = data[i]
            else:
                print(f"Error for {breed}")

            scraping_data.append(dict)
        else:
            print(f"There are NO data for {breed}")

    except AttributeError as s:
        print("AttributeError occured: ", s)

C:\Users\s4013\AppData\Local\Temp\ipykernel_32804\4274369560.py:14: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  origin = infobox.find('th', text='Origin').findNextSibling('td').text


AttributeError occured:  'NoneType' object has no attribute 'find'
There are NO data for Cocker_Spaniel
AttributeError occured:  'NoneType' object has no attribute 'findNextSibling'


In [8]:
# no infobox for Cocker_Spaniel
# no Origin for Japanese_Spaniel
scraping_data

[{'Breed': 'Affenpinscher',
  'Origin': 'Germany',
  'Height': ['23–30cm (9–12in)'],
  'Weight': ['3–6kg (7–13lb)'],
  'Coat': ['Wire'],
  'Color': ['Black, grey, silver, red, black and tan, or beige (mixed black, white, brown and red hairs), all with or without black mask'],
  'Littersize': ['2–3 puppies'],
  'Lifespan': ['11–14years']},
 {'Breed': 'American_Staffordshire_Terrier',
  'Origin': 'United States',
  'Height': ['16.9–18.8in (43–48cm)[1]'],
  'Weight': ['about 50–80lb (23–36kg)'],
  'Coat': ['Smooth'],
  'Color': ['black, fawn, red, white, blue, solid, part or patched, brindle (All-white, 80%+ white, black and tan, and liver are not encouraged). Must have black nose and not red.  Note there is also no such thing as a “blue nose Amstaff” or Merle colored Amstaff']},
 {'Breed': 'Australian_Terrier',
  'Origin': 'Australia',
  'Height': ['Dogs', '~25 centimetres (10in)'],
  '': ['Bitches', 'slightly less'],
  'Weight': ['Dogs', '~6.5 kilograms (14lb)'],
  'Lifespan': ['12-14 y